<a href="https://colab.research.google.com/github/gh243700/COMP0205/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sounddevice

In [ ]:
!apt-get install -y libportaudio2


In [ ]:
!pip install sounddevice

In [ ]:
!pip install faster_whisper

In [ ]:
!pip install vosk

In [ ]:
!pip install deep_translator

In [ ]:
!pip install gtts

In [ ]:
!wget https://alphacephei.com/vosk/models/vosk-model-small-en-us-0.15.zip
!unzip vosk-model-small-en-us-0.15.zip -d models/

--2025-05-19 08:45:19--  https://alphacephei.com/vosk/models/vosk-model-small-en-us-0.15.zip
Resolving alphacephei.com (alphacephei.com)... 188.40.21.16, 2a01:4f8:13a:279f::2
Connecting to alphacephei.com (alphacephei.com)|188.40.21.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41205931 (39M) [application/zip]
Saving to: ‘vosk-model-small-en-us-0.15.zip.1’

vosk-model-small-en 100%[===================>]  39.30M  12.8MB/s    in 3.1s    

2025-05-19 08:45:22 (12.8 MB/s) - ‘vosk-model-small-en-us-0.15.zip.1’ saved [41205931/41205931]

Archive:  vosk-model-small-en-us-0.15.zip
replace models/vosk-model-small-en-us-0.15/am/final.mdl? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!pip install ffmpeg-python

In [ ]:
!pip install gradio

In [ ]:
!pip uninstall -y whisper

In [ ]:
!pip install -U openai-whisper

In [ ]:
from google.colab import output
from IPython.display import Javascript, display

audio_queue = queue.Queue()
def save_audio(b64_audio):
    header, encoded = b64_audio.split(",", 1)
    audio_data = base64.b64decode(encoded)
    timestamp = int(time.time())
    filename = f"mic_chunk_{timestamp}.wav"
    with open(filename, "wb") as f:
        f.write(audio_data)
    print(f"🎤 저장됨: {filename}")
    audio_queue.put(filename)

def start_auto_recording():
    # 1) register under simple name
    output.register_callback("save_audio", save_audio)

    # 2) inject JS that calls that same name
    auto_record_js = """
    if (!window.autoRecorderRunning) {
      window.autoRecorderRunning = true;
      const sleep = t => new Promise(r => setTimeout(r, t));
      const toBase64 = blob => new Promise(r => {
        const reader = new FileReader();
        reader.onloadend = () => r(reader.result);
        reader.readAsDataURL(blob);
      });

      navigator.mediaDevices.getUserMedia({audio:true})  // must be in-cell
        .then(stream => {
          (async () => {
            while (true) {
              const recorder = new MediaRecorder(stream);
              const chunks = [];
              recorder.ondataavailable = e => chunks.push(e.data);
              recorder.start();
              await sleep(3000);
              recorder.stop();
              await new Promise(res => recorder.onstop = res);
              const blob = new Blob(chunks);
              const b64 = await toBase64(blob);
              google.colab.kernel.invokeFunction('save_audio', [b64], {});
            }
          })();
        })
        .catch(err => console.error("Mic permission error:", err));
    }
    """
    display(Javascript(auto_record_js))

⚠️ No file in audio_queue after 5s—waiting again.
⚠️ No file in audio_queue after 5s—waiting again.


In [ ]:
import os
import queue
import sounddevice as sd
import soundfile as sf
import numpy as np
from faster_whisper import WhisperModel, BatchedInferencePipeline
import string
from vosk import Model, KaldiRecognizer
from deep_translator import GoogleTranslator
from gtts import gTTS
import json
import threading
from threading import Thread
import concurrent.futures
import wave
import whisper
from IPython.display import Javascript, display
from google.colab import output
import base64
import time

# Settings
TRIGGER_WORD = "stop"
SAMPLE_RATE = 16000
MODEL_PATH = "models/vosk-model-small-en-us-0.15"
WHISPER_MODEL = "turbo"
MODEL = WhisperModel(WHISPER_MODEL, device="cuda", compute_type="int8")
batched_model = BatchedInferencePipeline(model=MODEL)

q = queue.Queue()
BUFFER_SIZE = 1024
buffer = [('', 0, '', 0) for _ in range(BUFFER_SIZE)]  # (text, flag, lang, duration_time)
b_index = 0
b_ptr = 0
lock = threading.Lock()
result_queue = queue.Queue()



def listen_for_trigger_and_save():
    print("🎧 Listening... Speak and end with 'STOP'")
    model = whisper.load_model("base")
    audio_data = []
    total_audio_data = []
    num = 0
    chunk_size = 8000

    # wait for first valid WAV chunk
    while True:
        try:
            file_path = audio_queue.get(timeout=5)
        except queue.Empty:
            print("⚠️ No file in audio_queue after 5s—waiting again.")
            continue

        print(f"📥 수신된 오디오 파일: {file_path}")
        try:
            wf = wave.open(file_path, "rb")
            break
        except wave.Error:
            print("❌ 파일이 WAV 포맷이 아님. 무시합니다.")

    recognizer = KaldiRecognizer(model, wf.getframerate())
    st = ''
    while True:
        data = wf.readframes(chunk_size)
        if not data:
            wf.close()
            os.remove(file_path)
            try:
                file_path = audio_queue.get(timeout=5)
            except queue.Empty:
                print("⚠️ No file in audio_queue after 5s—waiting again.")
                continue
            wf = wave.open(file_path, "rb")
            continue

        recognizer.AcceptWaveform(data)
        result = json.loads(recognizer.PartialResult())
        text = result.get("text", "")

        if text:
            total_audio_data.append(data)
            audio_data.append(data)
            st += text

            if TRIGGER_WORD in text.lower():
                print("✅ Trigger word detected.")
                break

            if st:
                audio = np.concatenate(audio_data)
                sf.write(f"input{num}.wav", audio, SAMPLE_RATE)
                duration_seconds = len(audio) / SAMPLE_RATE
                add_item(f"input{num}.wav", duration_seconds)
                audio_data = []
                st = ''
                num += 1

    full_audio = np.concatenate(total_audio_data)
    sf.write("input.wav", full_audio, SAMPLE_RATE)
    duration_seconds = len(full_audio) / SAMPLE_RATE
    print(f"⏱️ Recording duration: {duration_seconds:.2f} seconds")
    result_queue.put(("input.wav", duration_seconds))

def add_item(audio_path, duration_seconds):
    global b_index
    while buffer[b_index][1] != 0:
        continue
    buffer[b_index] = (audio_path, 1, '', duration_seconds)
    b_index = (b_index + 1) % BUFFER_SIZE

def translate_text(i):
    global MODEL
    audio_path, flag, lang, duration_seconds = buffer[i]
    with lock:
        segments, info = batched_model.transcribe(audio_path, beam_size=5, batch_size=16)

    text = ""
    for segment in segments:
        text += segment.text

    if any(char in text for char in "가나다라마바사"):
        translated = GoogleTranslator(source='ko', target='en').translate(text)
        lang = 'en'
    else:
        translated = GoogleTranslator(source='en', target='ko').translate(text)
        lang = 'ko'

    buffer[i] = (translated, 2, lang, duration_seconds)

def transRoutine():
    with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
        i = 0
        while True:
            if buffer[i][1] == 1:
                executor.submit(translate_text, i)
                i = (i + 1) % BUFFER_SIZE

def speak(text, lang):
    tts = gTTS(text=text, lang=lang)
    tts.save("output.mp3")
    os.system("afplay output.mp3" if os.name == "posix" else "start output.mp3")

def use_item_routine():
    global b_ptr
    while True:
        if buffer[b_ptr][1] == 2:
            text, flag, lang, duration_seconds = buffer[b_ptr]
            print(f"🔈 Speaking: {text}")
            speak(text, lang)
            buffer[b_ptr] = ('', 0, '', 0)
            b_ptr = (b_ptr + 1) % BUFFER_SIZE

def main():
    start_auto_recording()
    producer_thread = Thread(target=listen_for_trigger_and_save)
    translator_thread = Thread(target=transRoutine)
    speaker_thread = Thread(target=use_item_routine)

    producer_thread.start()
    translator_thread.start()
    speaker_thread.start()
    #audio_path, duration = result_queue.get()
    #print(f"🎤 Audio saved: {audio_path}")
    #print(f"⏱️ Total recording duration: {duration:.2f} seconds")
    producer_thread.join()

if __name__ == "__main__":
    main()
